### Imports:

In [25]:
import numpy as np
import random

### Generating Data:
* Define $ r \sim U(-50,50) $, $r_n \sim N(0,1)$, ground truth weight vector $w$ 
* Data: $x = r + r_n$
* Labels: $y = w' \cdot x$

In [15]:
"""
Inputs:
    w - the ground truth vector
    n - the number of data points to generate
Output:
    x - n x d matrix of data
    y - n x 1 array of labels for data
"""
def generate_data(w, n):
    d = len(w) # dimension of data
    rn = np.random.randn(n,d) # n x d matrix samples from N(0,1)
    r = np.random.rand(n,d)*100.0 - 50.0 # n x d matrix samples from U(-50,50)
    x = rn + r
    y = np.dot(w,x.T)
    return x,y

In [62]:
w = [1.0, 2.0, 3.0, 4.0]
n = 10000
x,y = generate_data(w,n)

In [63]:
def gradient(wi, xi, yi):
    return 2.0*xi*(np.dot(wi, xi) - yi)

### Implementing Regular SGD:

In [64]:
def sgd(alpha, x, y, iters):
    n,d = np.shape(x)
    wi = np.random.rand(d)
    for i in range(iters):
        idx = random.randint(0, n-1)
        xi = x[idx, :]
        yi = y[idx]
        grad = gradient(wi,xi,yi)
        wi = wi - alpha*(grad)
    return wi

In [72]:
weights_sgd = sgd(0.0001, x, y, 100)
print(weights_sgd)

[ 0.99999998  1.99999759  2.99999862  3.99999772]


### Implementing Regular SVRG:

In [105]:
def svrg(alpha, x, y):
    n,d = np.shape(x)
    w_0_tilde = np.random.rand(d) * 10 # init param
    m = 2*n # update frequency
    converged = False
    prev_obj = 0
    while not converged:
        w_tilde = w_0_tilde
        mu_tilde = np.zeros(d)
        # full gradient calculation
        for i in range(n):
            xi = x[i, :]
            mu_tilde += gradient(w_tilde, xi, y[i])
        mu_tilde = mu_tilde/(1.0*n)
        w_0 = w_tilde
        for t in range(0,m):
            i = random.randint(0, n-1)
            xi = x[i, :]
            yi = y[i]
            grad_1 = gradient(w_0,xi,yi)
            grad_2 = gradient(w_tilde,xi,yi)
            w_0 = w_0 - alpha*(grad_1 - grad_2 + mu_tilde)
            #stuff
        w_0_tilde = w_0
        obj = np.dot(w_0_tilde.T,x.T) - y
        obj = sum(obj * obj) / (1.0*n)
        if abs(prev_obj - obj) < 0.0001: # check convergence
             converged = True
        prev_obj = obj
    return w_0_tilde

In [106]:
weights_svrg = svrg(0.0001, x, y)
print(weights_svrg)

[ 0.99995193  2.00011606  2.99995524  4.00000076]
